## Initial Scraping
scrape categories and all books for each category and then insert into mongo db

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pymongo
import os

In [2]:
# browser = Browser('chrome')
html = "http://quotes.toscrape.com/"

In [3]:
# import requests
# page = requests.get(html)

In [ ]:
# def get_soup(url):
#     browser.visit(url)
#     html = browser.html    
#     soup = BeautifulSoup(html, 'html.parser')
#     return soup

In [ ]:
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
scraped = []
for quote in quotes:
    text = quote.find('span', class_='text').text
    author = quote.find('small', class_='author').text
    scraped.append([text, author])

In [ ]:
def get_quotetext():
    soup = BeautifulSoup(html, 'html.parser')
    quotes = []
    quotes_html = soup.find('span', class_='text')
    for quote in quotes_html:
        data = { 'text' : quote.text.strip()}
        quotes.append(data)
    return quotes

In [ ]:
quotes

In [ ]:
quotes = get_quotetext()

In [ ]:
for quote in quotes:
    print(quote)

In [ ]:
def get_articles():
    html = browser.html    
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('article', class_='product_pod')
    return articles

In [ ]:
def get_books_in_article(url , articles):
    books = []
    for article in articles:
        book ={
            'title' : article.h3.a.text,
            'href' : urljoin(url,article.h3.a['href'])
        }
        books.append(book)
    return books

In [ ]:
def scrape_all_books(url):
    more_books = True
    first_iterations = True
    iterations = 0
    books = []
    while more_books:
#         soup = get_soup(url)
        iterations += 1
        if first_iterations:
            browser.visit(url)
            first_iterations = False
        else:
            pass
        articles = get_articles()
        books_for_article = get_books_in_article(url, articles)
        books.append(books_for_article)
        #         if iterations > 3:
#             more_books=False
        more_books=True
        try:
            browser.links.find_by_partial_text('next').click()          
        except:
#             print("Scraping Complete")
            more_books = False
    return books

In [ ]:
categories = get_categories()

In [ ]:
for category in categories:
    print(f"Scrapping for {category['category']}....")
    books = scrape_all_books(category['href'])
    category['books'] = books

In [ ]:
browser.quit()

In [ ]:
def save_books(books):
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.books
    collection = db.books
    collection.insert_many(categories)


In [ ]:
save_books(categories)